In [2]:
! pip install litellm

In [78]:
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")

In [79]:
from litellm import completion
import os

## set ENV variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


# OpenAI Calls without function

In [80]:
openai_response = completion(
  model="openai/gpt-4o-mini",
  messages=[{ "content": "Hello, how are you?","role": "user"}]
)
print(openai_response.choices[0].message.content)

Hello! I'm just a virtual assistant, but I'm here and ready to help you. How can I assist you today?


# OpenAI Calls without function with streaming

In [63]:
openai__response = completion(
  model="openai/gpt-4o",
  messages=[{ "content": "Hello, how are you?","role": "user"}],
  stream=True,
)
print(openai__response)
for chunk in openai__response:
    print(chunk["choices"][0]["delta"]["content"])

Hello
!
 I'm
 just
 a
 program
,
 so
 I
 don't
 have
 feelings
,
 but
 I'm
 here
 and
 ready
 to
 help
 you
.
 How
 can
 I
 assist
 you
 today
?
None


## Openai parallel function calling without streaming

In [59]:
import litellm
import json
import requests
import os

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = litellm.completion(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
print("\nLLM Response1:\n", response)
response_message = response.choices[0].message
tool_calls = response.choices[0].message.tool_calls



## Parse the Model Response and Execute Functions


# Check if the model wants to call a function
if tool_calls:
    # Execute the functions and prepare responses
    available_functions = {
        "get_current_weather": get_current_weather,
    }

    messages.append(response_message)  # Extend conversation with assistant's reply

    for tool_call in tool_calls:
      print(f"\nExecuting tool call\n{tool_call}")
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
      # calling the get_current_weather() function
      function_response = function_to_call(
          location=function_args.get("location"),
          unit=function_args.get("unit"),
      )
      print(f"Result from tool call\n{function_response}\n")

      # Extend conversation with function response
      messages.append(
          {
              "tool_call_id": tool_call.id,
              "role": "tool",
              "name": function_name,
              "content": function_response,
          }
      )



final_fun_calling_response = litellm.completion(
    model="gpt-3.5-turbo-1106",
    messages=messages,
)
print("final_fun_calling_response\n", final_fun_calling_response.choices[0].message.content)


LLM Response1:
 ModelResponse(id='chatcmpl-AvjY9AnVv9N6GuLjPuLkplxliFrZF', created=1738323153, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_2f141ce944', choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"location": "San Francisco", "unit": "celsius"}', name='get_current_weather'), id='call_DpAQgIqnjmtSttv196OR1woj', type='function'), ChatCompletionMessageToolCall(function=Function(arguments='{"location": "Tokyo", "unit": "celsius"}', name='get_current_weather'), id='call_aumhYvcs3xWa51nFpuWcRbtj', type='function'), ChatCompletionMessageToolCall(function=Function(arguments='{"location": "Paris", "unit": "celsius"}', name='get_current_weather'), id='call_FhXR2DE8e88F8SMh3mIFsh8E', type='function')], function_call=None, provider_specific_fields={'refusal': None}, refusal=None))], usage=Usage(completion_tokens=78, prompt_tokens=88, tot

# Openai Parallel function calling with streaming

In [66]:
import litellm
import json
import requests
import os

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = litellm.completion(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
print("\nLLM Response1:\n", response)
response_message = response.choices[0].message
tool_calls = response.choices[0].message.tool_calls



## Parse the Model Response and Execute Functions


# Check if the model wants to call a function
if tool_calls:
    # Execute the functions and prepare responses
    available_functions = {
        "get_current_weather": get_current_weather,
    }

    messages.append(response_message)  # Extend conversation with assistant's reply

    for tool_call in tool_calls:
      print(f"\nExecuting tool call\n{tool_call}")
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
      # calling the get_current_weather() function
      function_response = function_to_call(
          location=function_args.get("location"),
          unit=function_args.get("unit"),
      )
      print(f"Result from tool call\n{function_response}\n")

      # Extend conversation with function response
      messages.append(
          {
              "tool_call_id": tool_call.id,
              "role": "tool",
              "name": function_name,
              "content": function_response,
          }
      )



final_fun_calling_response = litellm.completion(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    stream=True
)

print("final_fun_calling_response\n", final_fun_calling_response)
for chunk in final_fun_calling_response:
    print(chunk["choices"][0]["delta"]["content"])


LLM Response1:
 ModelResponse(id='chatcmpl-Avk8norxKqWr5kQI3OrV2cGMNYpTR', created=1738325425, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_2f141ce944', choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), id='call_24hlBVlyKAPQ8Xtevn8hbTuM', type='function'), ChatCompletionMessageToolCall(function=Function(arguments='{"location": "Tokyo, Japan", "unit": "celsius"}', name='get_current_weather'), id='call_9LvztpyqKP8LgOEYhivVAv0R', type='function'), ChatCompletionMessageToolCall(function=Function(arguments='{"location": "Paris, France", "unit": "celsius"}', name='get_current_weather'), id='call_GeFk8NZUBtJJMkWCeckK2ugO', type='function')], function_call=None, provider_specific_fields={'refusal': None}, refusal=None))], usage=Usage(completion_tokens=84, pr

# Anthropic without function

In [67]:
## set ENV variables

os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

anthropic_response = completion(
  model="anthropic/claude-2",
  messages=[{ "content": "what's the weather in SF","role": "user"}]
)
print(anthropic_response)
print(anthropic_response.choices[0].message.content)

ModelResponse(id='chatcmpl-90552026-f988-4e1c-ab70-5abcfa475faf', created=1738325481, model='claude-2', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop_sequence', index=0, message=Message(content=" Unfortunately, I do not have up-to-date access to weather conditions in specific locations. As an AI assistant without direct access to meteorological data sources, I cannot provide current weather details for San Francisco or other cities. I'd be happy to discuss other topics I can assist you with though!", role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=58, prompt_tokens=13, total_tokens=71, completion_tokens_details=None, prompt_tokens_details=None))
 Unfortunately, I do not have up-to-date access to weather conditions in specific locations. As an AI assistant without direct access to meteorological data sources, I cannot provide current weather details for San Francisco or other

# Anthropic without function with streaming

In [62]:
anthropic_response = completion(
  model="anthropic/claude-3-sonnet-20240229",
  messages=[{ "content": "Hello, how are you?","role": "user"}],
  stream=True
)
print(anthropic_response)
for chunk in anthropic_response:
    print(chunk["choices"][0]["delta"]["content"])

Hello
! As
 an AI
 language
 model, I don
't have physical sens
ations or emotions
,
 but I'm operating
 properly
 and ready
 to assist
 you with any questions
 or tasks you may
 have. How
 can
 I help
 you today?
None


# Anthropic with function

In [77]:

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]

# response = completion(
#     model="anthropic/claude-3-opus-20240229",
#     messages=messages,
#     tools=tools,
#     tool_choice="auto",
# )
# # Add any assertions, here to check response args
# print(response)
# assert isinstance(response.choices[0].message.tool_calls[0].function.name, str)
# assert isinstance(
#     response.choices[0].message.tool_calls[0].function.arguments, str
# )


response = completion(
    model="anthropic/claude-3-opus-20240229",
    messages=messages,
    tools=tools,
    tool_choice={"type": "function", "function": { "name": "get_current_weather" }}, # Updated tool_choice to force the function call
)
print(response)

# Check if the model wants to call a function
if tool_calls:
    # Execute the functions and prepare responses
    available_functions = {
        "get_current_weather": get_current_weather,
    }

    messages.append(response_message)  # Extend conversation with assistant's reply

    for tool_call in tool_calls:
      print(f"\nExecuting tool call\n{tool_call}")
      function_name = tool_call.function.name
      function_to_call = available_functions[function_name]
      function_args = json.loads(tool_call.function.arguments)
      # calling the get_current_weather() function
      function_response = function_to_call(
          location=function_args.get("location"),
          unit=function_args.get("unit"),
      )
      print(f"Result from tool call\n{function_response}\n")

      # Extend conversation with function response
      messages.append(
          {
              "tool_call_id": tool_call.id,
              "role": "tool",
              "name": function_name,
              "content": function_response,
          }
      )

final_fun_calling_response = litellm.completion(
    model="anthropic/claude-3-opus-20240229", # Added anthropic/ prefix to the model name
    messages=messages,
    tools=tools, # Added the tools parameter back into the final completion call
    stream=True
)
# print("final_fun_calling_response\n", final_fun_calling_response)
for chunk in final_fun_calling_response:
    print(chunk["choices"][0]["delta"]["content"])


ModelResponse(id='chatcmpl-dda7beea-901d-434e-9be2-1face69f0a54', created=1738326446, model='claude-3-opus-20240229', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=0, function=Function(arguments='{"location": "Boston, MA", "unit": "fahrenheit"}', name='get_current_weather'), id='toolu_01WiZ5MNHhkuDqjy2M59swKx', type='function')], function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=53, prompt_tokens=415, total_tokens=468, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0))

Executing tool call
ChatCompletionMessageToolCall(function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), id='call_2

In [69]:
response = completion(
    model="anthropic/claude-3-opus-20240229",
    messages=messages,
    tools=tools,
    tool_choice={"type": "tool", "name": "get_weather"},
)

Exception: Invalid tool choice, tool_choice={'type': 'tool', 'name': 'get_weather'}. Please ensure tool_choice follows the OpenAI spec

ModelResponse(id='chatcmpl-53163eb4-6bcb-43a5-b2b7-986b7188768a', created=1738326355, model='claude-3-opus-20240229', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(index=0, function=Function(arguments='{"location": "Boston, MA", "unit": "fahrenheit"}', name='get_current_weather'), id='toolu_01ACuoFccMAGhXZniTaGiBse', type='function')], function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=53, prompt_tokens=1772, total_tokens=1825, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0))

Executing tool call
ChatCompletionMessageToolCall(function=Function(arguments='{"location": "San Francisco, CA", "unit": "celsius"}', name='get_current_weather'), id='call